<img src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cian_main_theme.png" height="1000" width="1000"> 


## Первый парсер ЦИАН: моя подготовка к паре 

План обломов: 

- Качаем ссылки, начинают появляться дубликаты => максимум 54 страницы в выдаче
- Осторожнее с классами и поиском нужных элементов, можно накачать лишнего (но в данном слуае насрать)
- Обман в числе доступных объявлений (ещё n таких же квартир в этом доме ко ко ко ко ко) 

- Забан и капча 




__Основное:__ 

* Функция для формирования ссылок 
* Функция для скачки ссылок на объявления 
* Функция для скачки основной информации со сранички квартиры 

__Антизабан:__

* Функция, которая понимает, что вылезла капча 
* Функция, которая проверяет текущий IP адрес 
* Тор либо прокси смена IP при забане 
* Код, который собирает прокси с разных ссылок и оставляет хороший список (надо сделать 1 раз)


In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
from fake_useragent import UserAgent
UserAgent().chrome

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17'

In [3]:
import re
import time 
from tqdm import tqdm as tqdm_notebook

In [4]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

checkIP()

Current IP Address: 109.252.99.50


In [5]:
import socks
import socket
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socks.socksocket

checkIP()

Current IP Address: 185.220.101.62


In [6]:
for i in range(10):
    checkIP()
    time.sleep(5)

Current IP Address: 185.220.101.62
Current IP Address: 185.220.101.62
Current IP Address: 192.160.102.166
Current IP Address: 192.160.102.166
Current IP Address: 185.220.101.46
Current IP Address: 185.220.101.46
Current IP Address: 192.42.116.18
Current IP Address: 192.42.116.18
Current IP Address: 217.79.178.53
Current IP Address: 217.79.178.53


## 1. Генерация ссылок 

In [7]:
def create_url(params):
    main_page = 'https://www.cian.ru/cat.php?'
    params_1 = 'engine_version=2&deal_type=sale&offer_type=flat'
    params_2 = '&district={district}&{room}&region={region}&object_type={objt}&p='.format(**params)
    return main_page + params_1 + params_2

params = {
    'region': 1,
    'district': 1,
    'room': 'room1=1',
    'objt': 1}


districts = [1, 325, 326, 4, 5, 6, 7, 8, 9, 10, 11, 151]

rooms = ['room1=1', 'room2=1', 'room3=1', 'room4=1', 
         'room5=1', 'room6=1', 'room7=1', 'room9=1']

objt = [1,2]

In [8]:
main_urls = [ ]

for dis in districts:
    for rm in rooms:
        for ob in objt:
            params = {
                'region': 1,
                'district': dis,
                'room': rm,
                'objt': ob}
            
            main_urls.append(create_url(params))
            
len(main_urls)

192

## 2. Сбор ссылок

In [42]:
MAX_RETRIES = 10

session = requests.Session()
adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
session.mount('https://', adapter)
session.mount('http://', adapter)

In [88]:
def check_capcha(soup):
    if soup.title.text == 'Captcha - база объявлений ЦИАН':
        #print("Вылезла сраная капча!")
        return True
    else:
        return False

    
def get_cnt(url):
    """
        Сколько штук должно скачаться
    """
    
    response = session.get(url, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.content,'html.parser') 
    
    # проверка капчи
    is_capcha = check_capcha(soup)
    if is_capcha:
        time.sleep(10)
        print('Делаю новую попытку скачать')
        checkIP() # Чтобы видеть текущий IP 
        return get_cnt(url)
    else:
        cnt = soup.find('div', {'class': '_93444fe79c--totalOffers--22-FL'})
        if cnt is None:
            print('Нет вариков:', url)
            return 0
        else:
            cnt = cnt.text
            print(cnt)
            return int(re.sub('[^0-9]', '', cnt))

    
def get_hrefs(url, p):
    
    url = url + str(p)
    
    response = session.get(url, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.content,'html.parser') 
    
    # проверка капчи
    is_capcha = check_capcha(soup)
    if is_capcha:
        time.sleep(10)
#        print('Делаю новую попытку скачать :(')
#       checkIP() # Чтобы видеть текущий IP 
        return get_hrefs(url, p)
    else:
#        print('Удалось скачать!')
        flats = soup.find_all('div', {'class': '_93444fe79c--card--_yguQ'})
        hrefs = [item.find('a', {'class': 'c6e8ba5398--header--1fV2A'}).get('href') for item in flats]
        return hrefs

In [50]:
# Пример повторных запросов в рекурсии 

# def test_f(x):
#     x += 1
#     if x < 5:
#         return test_f(x)
#     else:
#         return x
    
# test_f(2)

In [82]:
# all_hrefs = { }

In [ ]:
# cur_url = main_urls[1]

In [81]:
!ls -lh

total 920
-rw-r--r--  1 fulyankin  staff    36K 23 янв 13:06 CIAN.ipynb
-rw-r--r--@ 1 fulyankin  staff   9,8K 22 янв 22:23 TorCrawler.py
drwxr-xr-x  3 fulyankin  staff   102B 22 янв 22:23 __pycache__
-rw-r--r--  1 fulyankin  staff   408K 23 янв 10:37 hrefs_1.pickle


In [59]:
import pickle 

with open('hrefs_2.pickle', 'wb') as f:
    pickle.dump(all_hrefs, f)

In [90]:
len(all_hrefs.keys())

2

In [91]:
sum([len(v) for v in all_hrefs.values()])

102

In [92]:
x = [ ]
for v in all_hrefs.values():
    x.extend(v)
len(set(x))

102

In [93]:
len(main_urls)

192

In [94]:
p, n_max

(1, 11)

In [77]:
cur_url = 'https://www.cian.ru/cat.php?engine_version=2&deal_type=sale&offer_type=flat&district=4&room6=1&region=1&object_type=2&p='


In [80]:
get_cnt(cur_url)

Делаю новую попытку скачать
Current IP Address: 199.249.230.64
Делаю новую попытку скачать
Current IP Address: 169.197.97.34
Делаю новую попытку скачать
Current IP Address: 176.10.99.200
Делаю новую попытку скачать
Current IP Address: 54.39.152.58
Делаю новую попытку скачать
Current IP Address: 109.70.100.24
19 предложений отсортированы По умолчанию


19

In [89]:
for cur_url in main_urls[58:100]:

    hrefs = [ ]

    # выяснили сколько делать запросов
    cnt = get_cnt(cur_url)
    n_max = (cnt + 28 - 1) // 28 + 1
    n_max = min(n_max, 54)

    # качаем ссылки по текущим объектам
    for p in range(1, n_max):
        time.sleep(5)
        cur_href = get_hrefs(cur_url, p)
        if len(cur_href) == 0:
            break
        hrefs.extend(cur_href)

    hrefs = list(set(hrefs))
    print('скачалось', len(hrefs), 'ссылок')
    all_hrefs[cur_url] = hrefs

Делаю новую попытку скачать
Current IP Address: 23.129.64.229
Делаю новую попытку скачать
Current IP Address: 199.249.230.101
Делаю новую попытку скачать
Current IP Address: 185.220.101.73
Делаю новую попытку скачать
Current IP Address: 185.220.101.70
Делаю новую попытку скачать
Current IP Address: 185.220.102.7
Делаю новую попытку скачать
Current IP Address: 172.96.118.42
Делаю новую попытку скачать
Current IP Address: 185.220.101.22
Делаю новую попытку скачать
Current IP Address: 104.244.76.245
Делаю новую попытку скачать
Current IP Address: 77.247.181.163
Делаю новую попытку скачать
Current IP Address: 192.68.11.219
Делаю новую попытку скачать
Current IP Address: 85.248.227.163
Делаю новую попытку скачать
Current IP Address: 162.247.74.202
Делаю новую попытку скачать
Current IP Address: 195.176.3.24
Делаю новую попытку скачать
Current IP Address: 195.206.105.217
Делаю новую попытку скачать
Current IP Address: 178.165.72.177
Делаю новую попытку скачать
Current IP Address: 5.199.135.1

KeyboardInterrupt: 

In [ ]:
p, n_max

In [ ]:
cur_url

In [ ]:
get_hrefs(cur_url, p)

In [ ]:
get_cnt(cur_url)

In [ ]:
main_page = 'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=1&p={}'.format(2)

response = requests.get(main_page, headers={'User-Agent': UserAgent().chrome})

soup = BeautifulSoup(response.content,'html.parser') 

for i in range(200):
    time.sleep(5)
    checkIP()
    if check_capcha(soup) == False:
        print('збс')
    else:
        print('хуйня')

In [ ]:
response = requests.get(cur_url)
soup = BeautifulSoup(response.content,'html.parser')
# cnt = soup.find('div', {'class': '_93444fe79c--totalOffers--22-FL'}).text
# int(re.sub('[^0-9]', '', cnt))

In [ ]:
soup.find('div', {'class': '_93444fe79c--totalOffers--22-FL'}) is None

In [ ]:
len(hrefs), len(set(hrefs)), 28*54, len(set(hrefs))//28

In [ ]:
28*5

In [ ]:
hrefs[-28*3:-28*2]

In [ ]:
cur_href

In [ ]:
main_urls[1]

In [ ]:
hrefs[-1]

In [ ]:
from collections import Counter
cnt = Counter(hrefs)
cnt.most_common()

In [ ]:
checkIP()

In [ ]:
checkIP()

In [ ]:
checkIP()

In [ ]:
import requests

proxies = {
    'http': '163.172.219.130:443',
    'https': '163.172.219.130:443'
}

r = requests.get('https://httpbin.org/ip', proxies=proxies)

print(r.json())

Эксперименты: 9, 142, 646

In [ ]:
len(hrefs)

Сбор информации по ссылкам.

In [ ]:
def get_house_info(url):
    
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
        
    data = { }
    
    data['href'] = url
    
    # Вытаскиваем цену на квартиру
    price = soup.find("span", {"itemprop": "price"}).text
    data['price'] = price

    # Вытаскиваем метро и расстояние до него
    metro = soup.find_all("li", {"data-name": "renderUnderground"})
    data['metro'] = [m.a.text for m in metro]
    data['metro_time'] = [m.span.text for m in metro]

    # Адрес квартиры 
    adress = soup.find("div", {"data-name": "Geo"}).find('span', {"itemprop": "name"})
    data['adress'] = adress.get("content")

    # Титульник объявления
    data['offer_title'] = soup.find('h1', {"data-name": "OfferTitle"}).text

    # Описание квартиры
    desc = soup.find('div', {"data-name": "Description"})
    data['description'] = desc.p.text

    for item in desc.find_all("div", {"class": "a10a3f92e9--info--2ywQI"}):
        jtem = item.find_all('div')
        data[jtem[-1].text] = jtem[0].text


    # координаты 
    a = [item for item in soup.find_all('script', {"type": "text/javascript"}) if 'coord' in str(item)]
    a = str(a[0])

    data['lat'] = re.findall('"lat":(\d\d.\d+)',a)[0]
    data['lng'] = re.findall('"lng":(\d\d.\d+)',a)[0]


    # первичка или вторичка, высота потолков
    house_inf_1 = soup.find("section", {"data-name": "AdditionalFeatures"})
    for item in house_inf_1.find_all("li"):
        jtem = item.find_all("span")
        data[jtem[0].text] = jtem[-1].text

    # тип дома и тип лифта
    house_inf_2 = soup.find("div", {"data-name": "BtiHouseData"})
    for item in house_inf_2.find_all("div", {"class": "a10a3f92e9--item--2Ig2y"}):
        jtem = item.find_all("div")
        data[jtem[0].text] = jtem[-1].text
        
    return data

In [ ]:
data = [ ]

i = 0
for url in hrefs:
    time.sleep(0.3)
    data.append(get_house_info(url))
    i+=1
    print(i)

In [ ]:
len(data)

In [ ]:
url

In [ ]:
import pandas as pd
df = pd.DataFrame(data)

In [ ]:
df.shape

In [ ]:
hrefs[-1]

In [ ]:
# Вытаскиваем цену на квартиру
# Вытаскиваем метро
# Вытаскиваем количество комнат в квартире
# Вытаскиваем расстояние до метро то ли пешком то ли на машине...
# Вытаскиваем координаты квартиры
# Вытащили все строки
# очищаем каждую строку! 
# запоминаем

In [ ]:
Площадь комнат:	17.21 м2
Координаты	[37.774751,55.670397]
Этаж:	15 / 17
Метро	Люблино,
Вид из окна:	двор
Комнаты	1-комн. кв. \n \n\n \n ...
Сдача ГК:	2 кв. 2019 года
Общая площадь:	24,1 м2
Совмещенных санузлов:	1
Тип продажи:	свободная
Лифт:	1 пасс. + 1 груз.
До метро	4\n мин.\n \n ...
Балкон:	1 лодж.
Жилая площадь:	17,2 м2
Тип дома:	новостройка, \n монолитный
Цена	3 677 660 руб.
Площадь кухни:	–

Балкон:	Вид из окна:	Высота потолков:	Год постройки:	До метро	Жилая площадь:	Комнаты	Координаты	Лифт:	Метро	...	Санузел:	Сдача ГК:	Совмещенных санузлов:	Статус помещения:	Телефон:	Тип дома:	Тип продажи:	Цена	Этаж:	Хрефы

<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/5.jpg" height="700" width="700">


* https://medium.com/@neurohiveru/%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%80-%D0%B2%D0%B7%D0%BB%D0%BE%D0%BC%D0%B0-%D0%BF%D1%80%D0%BE%D1%81%D1%82%D0%BE%D0%B9-%D0%BA%D0%B0%D0%BF%D1%87%D0%B8-%D0%B7%D0%B0-15-%D0%BC%D0%B8%D0%BD%D1%83%D1%82-%D1%81-%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E-%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F-597d2b51a75a




https://lenta.ru/articles/2017/03/14/captcha/

In [ ]:
from TorCrawler import TorCrawler
# Создаём свой краулер, в опциях вводим пароль 
crawler = TorCrawler(ctrl_pass='mypassword') 

In [ ]:
crawler.ip

In [ ]:
crawler.n_requests

In [ ]:
crawler.rotate()

In [ ]:
meme_page = 'http://knowyourmeme.com/memes/doge'
response = crawler.get(meme_page, headers={'User-Agent': UserAgent().chrome})
type(response)

In [ ]:
views = response.find('dd', attrs={'class':'views'})
views

https://habr.com/ru/post/442034/